In [1]:
import cv2
import dlib
import os
import csv
import numpy as np
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
# PATH TO ALL IMAGES
#basedir, image_paths
basedir = '../Datasets/cartoon_set'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

In [ ]:
#Use Hough Transform to find eye circles in an image
def detect_eye(image_file)
    # Read image as gray-scale
    img = cv2.imread('2.png', cv2.IMREAD_COLOR)
    # Convert to gray-scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #test
    # Blur the image to reduce noise
    img_blur = cv2.medianBlur(gray, 5)
    # Apply hough transform on the image
    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, img.shape[0]/2, param1=200, param2=10, minRadius=15, maxRadius=20)
    # Draw detected circles
    detected_eye = circles
    if circles is not None: 
        # Extract eyes
            for (x, y, r) in detected_eye: #get coordinates and size of a circle containing an eye
                #print(f"eyes found in file: {image_file}") 
                gray = gray[y:y+r, x:x+r] #Cut the frame to size

    #Draws circles around the eye  - FOR TEST ONLY, NOT NEEDED FOR THE MODEL          
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw outer circle
            cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # Draw inner circle
            cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)
    cv2.imshow('3', img)
    cv2.waitKey(0)

In [40]:
#create a function converting colours into hues. 
#these are already hues
BLUE = 137
GREEN = 57
BROWN = 11
BLACK = 160
GREY = 110

color = GREEN
hue = color // 2

lower_range = np.array([max(0, hue - 10), 0, 0], dtype=np.uint8)
upper_range = np.array([min(59, hue + 10), 255, 255], dtype=np.uint8)

#convert images into HSV
img = cv2.imread("3.png")
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#Apply mask to HSV images, and then save images 
mask = cv2.inRange(img_hsv, lower_range, upper_range)

binary_img = cv2.bitwise_and(img_hsv, img_hsv, mask=mask)
binary_img = cv2.cvtColor(binary_img, cv2.COLOR_BGR2GRAY)
#_, binary_img = cv2.threshold(binary_img, 127, 255, cv2.THRESH_BINARY)

cv2.imshow('3', binary_img)
cv2.waitKey(0)

-1

In [39]:
def extract_features_labels():
    """
    This function extracts the eye color features for all images in the folder 'Datasets/cartoon_set'.
    It also extracts the color label for each image and returns:
        HSV_features:  an array containing 3 points for each eye circle that was detected
        shape_labels:      an array containing 5 face shape labels (0,1,2,3,4) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    #shape labels from eye_color column [1] in the labels file
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    shape_labels = dict()
    for line in lines[1:]:
        columns = line.split('\t')
        shape_labels[columns[0]] = int(columns[1])
    
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            #print(image_path)
            file_name = img_path.split('.')[1].split('\\')[-1]
            #print(file_name)

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
                        
            #TODO: set up features taken from a new function: features, _ = run_dlib_shape(img)
            
            if features is not None:
                all_features.append(features)
                all_labels.append(shape_labels[file_name])

    #TODO: amend the following based on the new function: landmark_features = np.array(all_features)
    shape_labels = (np.array(all_labels)) # keeps the 5 eye color labels, 0, 1, 2, 3 and 4
    return landmark_features, shape_labels

-1

In [ ]:
def get_data():

    X, Y = extract_features_labels()
    Y = np.array([Y, -(Y - 1)]).T
    # Split into a training set and a test set using a stratified k fold
    # Split into a training and testing set

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
    return X_train, Y_train, X_test, Y_test
# sklearn functions implementation

#Set up SVM classifier
def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel='linear', C = 1.0)
    classifier.fit(training_images, training_labels)
    pred = classifier.predict(test_images)
    print("Accuracy:", accuracy_score(test_labels, pred))

    #print(pred)
    return pred

X_train, Y_train, X_test, Y_test = get_data()
num_X_test = len(X_test)
print(f'num_X_test{num_X_test}')
num_X_train = len(X_train)
print(f'num_X_train{num_X_train}')#
#REDO: the prediction based on the correct number of features in the feature list:

pred = img_SVM(X_train.reshape((num_X_train, 68*2)), list(zip(*Y_train))[0], X_test.reshape((num_X_test, 68*2)), list(zip(*Y_test))[0])